In [19]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import pandas as pd
from dhlab.nbtokenizer import tokenize
import jsonlines
dh.css()

# Lemmatize a set

a = dh.totals(100000)
lemma = dh.WordLemma(list(a.index)).lemmas
reduced_set = pd.DataFrame(lemma.merge(a.reset_index(), left_on=0, right_on='index').groupby(2)['freq'].max())
lower_set = pd.DataFrame(reduced_set.groupby(lambda x: x.lower())['freq'].sum())
lower_set.sort_values(by='freq', ascending=False)

# Generate sentences

In [41]:
a = api.word_concordance(dhlabid=list(corpus.frame.sample(10).dhlabid), words = ['arbeid'])

In [50]:
def show_concs(a):
    res = []
    for i, x in a.iterrows():
        try:
            res.append(' '.join([x['before'], x['target'], x['after']]))
        except:
            pass
    return res

In [52]:
#show_concs(a)

In [33]:
def generate_word_sequences_from_corpus(corpus, samples, words = None, limit=1000):
    sentences = api.word_concordance(dhlabid=list(corpus.sample(samples).dhlabid), words = words, before = 12, after = 12, limit = limit)
    a = sentences.show(style=False, n=limit).concordance.apply(lambda x:x.replace('<b>','').replace('</b>','')).values
    return a

In [66]:
def generate_word_sequences_from_corpus_wkonk(corpus, samples, words = None, limit=1000):
    sentences = api.word_concordance(dhlabid=list(corpus.sample(samples).dhlabid), words = words, before = 24, after = 0, limit = limit)

    return show_concs(sentences)

In [15]:
def convert(sentence, lower=False):
    """Tokenize i a string, turn all tokens to lower case and 
    remove punctuations and other stuff not normally part of a word"""
    lst = tokenize(sentence)
    lowered = [x for x in lst if x.isalpha()]
    return lowered

## Select corpus

In [59]:
corpus = dh.Corpus(doctype='digavis',  from_year=1978, to_year=1984, limit = 250000)


In [61]:
len(corpus.frame)

176522

,dhlabid,urn,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp
0,200515140,URN:NBN:no-nb_digavis_finnmarken_null_null_198...,finnmarken,None,None,None,None,Vadsø,19801018,1980,None,None,None,None,None,None,digavis,nb,20060101
1,203367129,URN:NBN:no-nb_digavis_ofotenstidende_null_null...,ofotenstidende,None,None,None,None,Narvik,19791229,1979,None,None,None,None,None,None,digavis,nb,20060101
2,200070117,URN:NBN:no-nb_digavis_akershusamtstidende_null...,akershusamtstidende,None,None,None,None,Frogn,19810128,1981,None,None,None,None,None,None,digavis,nb,20060101
3,203215398,URN:NBN:no-nb_digavis_stavangeraftenblad_null_...,stavangeraftenblad,None,None,None,None,Stavanger,19780429,1978,None,None,None,None,None,None,digavis,nb,20060101
4,202820298,URN:NBN:no-nb_digavis_sunnmorsposten_null_null...,sunnmorsposten,None,None,None,None,Ålesund,19810312,1981,None,None,None,None,None,None,digavis,nb,20060101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176517,201319625,URN:NBN:no-nb_digavis_ofotenstidende_null_null...,ofotenstidende,None,None,None,None,Narvik,19810725,1981,None,None,None,None,None,None,digavis,nb,20060101
176518,201059843,URN:NBN:no-nb_digavis_morgenbladet_null_null_1...,morgenbladet,None,None,None,None,Oslo,19810130,1981,None,None,None,None,None,None,digavis,nb,20060101
176519,200277399,URN:NBN:no-nb_digavis_bygdanytt_null_null_1980...,bygdanytt,None,None,None,None,Bergen,19800701,1980,None,None,None,None,None,None,digavis,nb,20060101
176520,203105745,URN:NBN:no-nb_digavis_haugesundsavis_null_null...,haugesundsavis,None,None,None,None,Haugesund,19820204,1982,None,None,None,None,None,None,digavis,nb,20060101


In [63]:
aften = corpus.frame[corpus.frame.title.str.contains('aftenposten')]
len(aften)

4434

## Generate json lines

%%time
for i in range(400):
    if i%100 == 0:
        print(i)
    a = generate_word_sequences_from_corpus(aften, 100, words = ['Alta-saken'], limit=1000)
    with open(f"extract_digavis_aftenpost_78_84_Alta.txt",mode='a') as wr:
        for sent in map(convert, a):
            wr.write(' '.join(sent))
            wr.write('\n')
    

In [67]:
%%time
with open(f"extract_digavis_aftenpost_78_84_Alta.txt",mode='w') as wr:
    for i in range(4):
        if i%100 == 0:
            print(i)
        a = generate_word_sequences_from_corpus_wkonk(aften, 100, words = ['.'], limit=1000)
        for sent in a:
            wr.write(sent)
            wr.write('\n')
    

0
CPU times: user 2.67 s, sys: 190 ms, total: 2.86 s
Wall time: 1min 17s
